In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

# ----------------------
# Define LeNet-5
# ----------------------
class LeNet5(nn.Module):
    def __init__(self):
        super(LeNet5, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, kernel_size=5, stride=1)
        self.pool1 = nn.AvgPool2d(kernel_size=2, stride=2)
        
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5, stride=1)
        self.pool2 = nn.AvgPool2d(kernel_size=2, stride=2)
        
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)  # 10 classes (digits)
    
    def forward(self, x):
        x = F.tanh(self.conv1(x))   # Original LeNet used Tanh, not ReLU
        x = self.pool1(x)
        
        x = F.tanh(self.conv2(x))
        x = self.pool2(x)
        
        x = x.view(-1, 16 * 5 * 5)  # Flatten
        x = F.tanh(self.fc1(x))
        x = F.tanh(self.fc2(x))
        x = self.fc3(x)
        return x

# ----------------------
# Data Loading (MNIST)
# ----------------------
transform = transforms.Compose([
    transforms.Pad(2),      # MNIST is 28x28, pad to 32x32
    transforms.ToTensor(),
])

train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1000, shuffle=False)

# ----------------------
# Training
# ----------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = LeNet5().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        
        if batch_idx % 100 == 0:
            print(f"Train Epoch: {epoch} [{batch_idx*len(data)}/{len(train_loader.dataset)}] "
                  f"Loss: {loss.item():.6f}")

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
    
    test_loss /= len(test_loader.dataset)
    accuracy = 100. * correct / len(test_loader.dataset)
    print(f"Test set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} "
          f"({accuracy:.2f}%)")

# ----------------------
# Run Training & Testing
# ----------------------
for epoch in range(1, 5):  # train for 4 epochs (you can increase)
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)


100%|██████████| 9.91M/9.91M [00:19<00:00, 507kB/s] 
100%|██████████| 28.9k/28.9k [00:00<00:00, 102kB/s]
100%|██████████| 1.65M/1.65M [00:02<00:00, 557kB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 4.54MB/s]


Train Epoch: 1 [0/60000] Loss: 2.300591
Train Epoch: 1 [6400/60000] Loss: 0.513367
Train Epoch: 1 [12800/60000] Loss: 0.144087
Train Epoch: 1 [19200/60000] Loss: 0.328213
Train Epoch: 1 [25600/60000] Loss: 0.170922
Train Epoch: 1 [32000/60000] Loss: 0.299810
Train Epoch: 1 [38400/60000] Loss: 0.271504
Train Epoch: 1 [44800/60000] Loss: 0.283090
Train Epoch: 1 [51200/60000] Loss: 0.203408
Train Epoch: 1 [57600/60000] Loss: 0.151354
Test set: Average loss: 0.0001, Accuracy: 9633/10000 (96.33%)
Train Epoch: 2 [0/60000] Loss: 0.134766
Train Epoch: 2 [6400/60000] Loss: 0.056544
Train Epoch: 2 [12800/60000] Loss: 0.106133
Train Epoch: 2 [19200/60000] Loss: 0.081711
Train Epoch: 2 [25600/60000] Loss: 0.122493
Train Epoch: 2 [32000/60000] Loss: 0.106875
Train Epoch: 2 [38400/60000] Loss: 0.120969
Train Epoch: 2 [44800/60000] Loss: 0.064271
Train Epoch: 2 [51200/60000] Loss: 0.057546
Train Epoch: 2 [57600/60000] Loss: 0.075199
Test set: Average loss: 0.0001, Accuracy: 9761/10000 (97.61%)
Train 